# set query and parameter u

In [1]:
import os
import glob
from collections import defaultdict
import re
import math
u = 1500.0

query = ["carbon emission", 
            "cutting carbon emission", 
            "flexible dieting", 
            "flexible dieting and vegetarianism", 
            "information of pest control in greenhouse", 
            "pest control greenhouse", 
            "greenhouse apples apple", 
            "green house apples apple"]
query_list = []
for item in query:
    query_list.append(item.split())

# get corpus statistics: corpus size and token frequency

In [2]:
file_list = glob.glob(os.path.join(os.getcwd(),'corpus', '*.txt' ))

In [3]:
corpus = {}
i=0
for file_path in file_list:
    with open(file_path, encoding= 'utf-8') as file_input:
           #created a dictionary of corpus with filenames as keys and content as value
        #corpus[filenames[i]]= (file_input.read())
        corpus[os.path.basename(file_path)]= (file_input.read())
        #print(corpus[os.path.basename(file_path)])
        #i +=1
#print(type(corpus))
#print(corpus["Observations_Concerning_the_Increase_of_Mankind_Peopling_of_Countries_etc..txt"])

In [4]:
# get the frequency of word in corpus
wordcount = defaultdict(int)
for key,value in corpus.items():
    for words in value.split():
        if words not in wordcount:
            wordcount[words] = 1
        else:
            wordcount[words] += 1
#print(wordcount)

In [5]:
corpus_size = sum(wordcount.values())

# Run the LM_Dirichlet

In [7]:

rank_score = {}
query_score = {}
query_frequency = {}
doc_size = {}
query_lm = {}
num = 0

for i in query_list:
    print(i)
    query_relative_doc = []
    doc_site_each_query = []
    for query_item in i:
        print(query_item)
        find_docID = False
        with open("unigramIDF.txt", "r") as mytxt:
            for lines in mytxt:      
                key,value = lines.split(' : ')
                if key == query_item: 
                    find_docID = True
                    for doc_pattern in re.findall(r'\((.[^\)\']+\.txt,\d+)\)', value):
                        query_relative_doc.append(doc_pattern)               
                    
                    for docId_frequency in query_relative_doc:
                        #print(docId_frequency)
                        mylist = docId_frequency.split(",")
                        #print(mylist)
                        #query_frequency["query_item"] = query_item
                        query_frequency[mylist[0]] = mylist[1]
                        doc_site_each_query.append(mylist[0])
                        if mylist[0] not in doc_size.keys():
                            doc_size[mylist[0]] = len(corpus[mylist[0]].split())
                    query_score[query_item] = query_frequency
        if find_docID == False:
            query_score[query_item] = "Empty"  
    f_score = {}

    for sdoc in doc_site_each_query:
        LM_score = 0
        for squery in i:
            #print(doc)
            if wordcount[squery] != 0:
                if query_score[squery] != "Empty":
                #print(query_score[squery])
                    if sdoc not in query_score[squery].keys():
                        f_score[sdoc] = 0
                    else:
                        f_score[sdoc] = query_score[squery][sdoc]

                else:
                    f_score[sdoc] = 0
                #print(f_score[sdoc],u * (wordcount[squery]/corpus_size),doc_size[sdoc])
                q_score = math.log((int(f_score[sdoc]) + u * (wordcount[squery]/corpus_size))/(doc_size[sdoc] + u))
            else:
                q_score = 0
            LM_score += q_score
            #print(LM_score)
        rank_score[sdoc] =  LM_score
        
  
    sorted_by_value = sorted(rank_score.items(), key=lambda kv: kv[1],reverse=True)
    
    write_path = os.getcwd() + "/task2" 
    filename = os.path.join(write_path, query[num] + ".txt")
    f = open(filename,"w+",encoding="utf-8",newline='\n')
    count = 1 
    for top_100 in sorted_by_value:
        if count < 101:
            f.write("%d %s %f\n" %(count,top_100[0],top_100[1]))
            count = count+1
        else:
            break
    f.close()
    
    f = open("top100Task2.txt","a+",encoding="utf-8",newline='\n')
    count = 1 
    #Format: query_id Q0 doc_id rank LMDirichlet_score system_name
    constant = "Q0"
    system_name = "Unigram_LMDirichlet_MingmingLi"
    for top_100 in sorted_by_value:
        if count < 101:
            f.write("%d %s %s %s %f %s\n" %(num+1,constant,top_100[0],count,top_100[1],system_name))
            count = count+1
        else:
            break
    f.close() 
    num = num + 1

['carbon', 'emission']
carbon
emission
['cutting', 'carbon', 'emission']
cutting
carbon
emission
['flexible', 'dieting']
flexible
dieting
['flexible', 'dieting', 'and', 'vegetarianism']
flexible
dieting
and
vegetarianism
['information', 'of', 'pest', 'control', 'in', 'greenhouse']
information
of
pest
control
in
greenhouse
['pest', 'control', 'greenhouse']
pest
control
greenhouse
['greenhouse', 'apples', 'apple']
greenhouse
apples
apple
['green', 'house', 'apples', 'apple']
green
house
apples
apple


In [11]:
#badk up function: get the frequency of query_word in specific document
def get_frequency(query_word,docID):
    #print(docID)
    frequency = 0
    doc_len = 0
    #print(corpus[docID])
    for word in corpus[docID].split():
        doc_len += 1
        if word == query_word:
            frequency += 1
            
    return frequency,doc_len
f,l = get_frequency("carbon","2009_United_Nations_Climate_Change_Conference.txt")
#print(f)
#print(l)